importing libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam

2024-04-08 23:54:03.031716: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 23:54:25.825092: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 23:54:52.134694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.__version__ 


'2.13.1'

Defining data Generators

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

Test data generators

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    '/home/alice/Desktop/Sign-Language-To-Text-Conversion/dataSet/trainingData',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 12453 images belonging to 27 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    '/home/alice/Desktop/Sign-Language-To-Text-Conversion/dataSet/testingData',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 4268 images belonging to 27 classes.


Load pre-trained ResNet50 model

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 [==============================] - 25s 0us/step


Add custom layers on top of the ResNet50

In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(27, activation='softmax')(x)

resnet_model = Model(inputs=base_model.input, outputs=predictions)

Freeze base layers

In [9]:
for layer in base_model.layers:
    layer.trainable = False

resnet_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

Train model

In [10]:
resnet_history = resnet_model.fit(train_generator, 
                                  epochs=5, 
                                  validation_data=test_generator)

Epoch 1/5


2024-04-09 00:02:03.501653: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-04-09 00:02:03.522741: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 20313600 exceeds 10% of free system memory.
2024-04-09 00:02:03.651279: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-04-09 00:02:06.091598: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-04-09 00:02:07.435545: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 106463232 exceeds 10% of free system memory.


390/390 [==============================] - 4345s 11s/step - loss: 1.2710 - accuracy: 0.6435 - val_loss: 0.3890 - val_accuracy: 0.9178
Epoch 2/5
390/390 [==============================] - 4428s 11s/step - loss: 0.3393 - accuracy: 0.9118 - val_loss: 0.1682 - val_accuracy: 0.9604
Epoch 3/5
390/390 [==============================] - 4329s 11s/step - loss: 0.1832 - accuracy: 0.9566 - val_loss: 0.1289 - val_accuracy: 0.9538
Epoch 4/5
390/390 [==============================] - ETA: 0s - loss: 0.1289 - accuracy: 0.9663

compare performance

In [ ]:
print("ResNet50 Model:")
print(resnet_history.history)